In [8]:
# En este ejemplo pueden haber cosas que no anden en el notebook pero sí en el binder
import ipywidgets as widgets
from histogramas import apply_y_function_transformation, apply_square, apply_sqrt, apply_lineal_by_parts_to_pixel
from PIL import Image
import io
import numpy as np
from IPython.display import FileLink, HTML
from datetime import datetime
import os

# Manipulación de Luminancia

In [2]:
image = widgets.Image(
    value=open('data/upload.png', 'rb').read(),
    width=800,
)
info_text = widgets.Label()
display(image)
display(info_text)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\x80\x00\x00\x01\x90\x08\x06\x00\x00\x00>\xf3\xd1…

Label(value='')

In [3]:
file_upload = widgets.FileUpload()
def handle_change(change):
    image.value = file_upload.data[0]

file_upload.observe(handle_change, names='data')

display(file_upload)

FileUpload(value={}, description='Upload')

In [4]:
# Aplicar filtro cuadrado
apply_square_filter_button = widgets.Button(description="Aplica cuadrado")
apply_sqr_root_filter_button = widgets.Button(description="Aplica raiz")
apply_by_parts_button = widgets.Button(description="Aplicar lineal por partes")
lower_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.001,
    description='Umbral inferior:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)
upper_slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.001,
    description='Umbral superior:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

def get_by_parts_args():
    return {
        'lower': lower_slider.value,
        'upper': upper_slider.value
    }

def generate_handler_with_args(function, args_function):
    
    def f(button):
        args = args_function()
        apply_f = function(**args)
        generate_handler(apply_f)(button)
    
    return f

def generate_handler(function):
    def _click_handler(button):
        info_text.value="Procesando..."
        
        # Convert to RGB since we're not interested in the alpha channel
        input_img = np.array(Image.open(io.BytesIO(image.value)).convert("RGB"))

        # Transformed image
        output_image = apply_y_function_transformation(input_img, function)

        # Generate PIL Image object 
        output_image_pil = Image.fromarray(output_image.astype(np.uint8))

        fp = io.BytesIO()

        output_image_pil.save(fp, format=image.format)

        # Show generated image
        image.value = fp.getvalue()
        info_text.value="Listo!"
    
    return _click_handler
    
apply_square_filter_button.on_click(generate_handler(apply_square))
apply_sqr_root_filter_button.on_click(generate_handler(apply_sqrt))
apply_by_parts_button.on_click(generate_handler_with_args(apply_lineal_by_parts_to_pixel,get_by_parts_args))

In [5]:
power_buttons = widgets.HBox([apply_sqr_root_filter_button, apply_square_filter_button])
by_parts_widgets = widgets.HBox([lower_slider, upper_slider, apply_by_parts_button])

In [6]:
widgets.VBox([power_buttons, by_parts_widgets], disabled=True)

In [7]:
# Guardar imágen
export_button = widgets.Button(description="Guardar imágen")
download_link = widgets.HTML()
def handle_export_button_click(button):

    timestamp = datetime.strftime(datetime.now(), "%Y%m%d%H%M%S")
    full_name = f"data/export/export-{timestamp}.{image.format}"
    file_name = full_name.split('/')[-1]
    
    if not os.path.isdir('data/export'):
        os.mkdir('data/export')
        
    fh = open(full_name, 'wb')
    
    fh.write(image.value)
    
    download_link.value = f'<a download="{file_name}" href="{full_name}">descargar</a>'

export_button.on_click(handle_export_button_click)
display(export_button)
display(download_link)

Button(description='Guardar imágen', style=ButtonStyle())

HTML(value='')